In [1]:
from o2s.models.object_2_sphere import Mesh2Sphere
from o2s.datasets.radar_dataset import RadarDataset

import numpy as np
import matplotlib.pyplot as plt
from torch_harmonics import plotting
from torch_geometric.loader import DataLoader
import xarray
import torch
from torch import nn

In [2]:
dataset = RadarDataset('/home/colin/hdd/workspace/datasets/test_dataset.nc', 'test')

In [3]:
B = 1
dataloader = DataLoader(dataset, batch_size=B, shuffle=True)

In [4]:
model = Mesh2Sphere(2, 2, 128, 0.5, use_mlp=False).cuda()

/home/colin/anaconda3/envs/torch/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/colin/anaconda3/envs/torch/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/colin/anaconda3/envs/torch/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/

In [7]:
batch, target = next(iter(dataloader))

In [8]:
model(batch.cuda())

tensor([[ 1.8118, -0.2371,  0.0936,  0.0538, -0.2180, -0.4480,  0.6724,  0.0948,
          0.6146, -1.0873, -0.0746,  1.0579,  0.2469,  0.2840, -0.0595, -0.0325,
          0.0870,  0.0689]], device='cuda:0', grad_fn=<ViewBackward0>)
tensor([[ 1.8118,  0.0936,  0.0538,  0.6724,  0.0948,  0.6146, -0.0746,  0.2469,
          0.2840,  0.0870,  0.0689]], device='cuda:0', grad_fn=<CatBackward0>)


RuntimeError: einsum(): the number of subscripts in the equation (2) is more than the number of dimensions (1) for operand 0

In [ ]:
from e3nn.o3 import Irreps, Irrep

In [ ]:
Irreps([(1, (l,1)) for l in range(3)] + [(1, (l,-1)) for l in range(3)])